# COVID-19 Novel Coronavirus: Infection Rate

## Background

Thanks to https://www.kaggle.com/khoongweihao for starting the oringinal notebook.

Also thanks to https://www.kaggle.com/davidbnn92 for showing how to load weather data

With this notebook, I plan to show the rate of change for US compared to the target rate of change (flattening the curve).

I will also incorporate weather data to see if it coorliates to the daily rate of change for certain regions.

The 2019 Novel Coronavirus, or 2019-nCoV, is a new respiratory virus first identified in Wuhan, Hubei Province, China. A novel coronavirus (nCoV) is a new coronavirus that has not been previously identified. The 2019 novel coronavirus (2019-nCoV), is not that same as the coronaviruses that commonly circulate among humans and cause mild illness, like the common cold.

A diagnosis with coronavirus 229E, NL63, OC43, or HKU1 is not the same as a 2019-nCoV diagnosis. These are different viruses and patients with 2019-nCoV will be evaluated and cared for differently than patients with common coronavirus diagnosis. 

Public health officials and partners are working hard to identify the source of the 2019-nCoV. Coronaviruses are a large family of viruses, some causing illness in people and others that circulate among animals, including camels, cats and bats. Analysis of the genetic tree of this virus is ongoing to know the specific source of the virus. SARS, another coronavirus that emerged to infect people, came from civet cats, while MERS, another coronavirus that emerged to infect people, came from camels. More information about the source and spread of 2019-nCoV is available on the 2019-nCoV Situation Summary: Source and Spread of the Virus.

This virus probably originally emerged from an animal source but now seems to be spreading from person-to-person. It’s important to note that person-to-person spread can happen on a continuum. Some viruses are highly contagious (like measles), while other viruses are less so. At this time, it’s unclear how easily or sustainably this virus is spreading between people. Learn what is known about the spread of newly emerged coronaviruses.

**Reference:** https://www.cdc.gov/coronavirus/2019-ncov/faq.html

In [ ]:
!pip install --upgrade -v pandas==1.0.1 --quiet

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import plotly.graph_objects as go
from fbprophet import Prophet
import pycountry
import plotly.express as px

from google.cloud import bigquery

print('Pandas version: {}'.format(pd.__version__))

# Data Import, Preprocessing and EDA

In [ ]:
def load_weather_data():
    %%time
    bq_assistant = BigQueryHelper("bigquery-public-data", "github_repos.noaa_gsod")

    client = bigquery.Client()
    dataset_ref = client.dataset("noaa_gsod", project="bigquery-public-data")
    dataset = client.get_dataset(dataset_ref)

    tables = list(client.list_tables(dataset))

    table_ref = dataset_ref.table("stations")
    table = client.get_table(table_ref)
    stations_df = client.list_rows(table).to_dataframe()

    table_ref = dataset_ref.table("gsod2020")
    table = client.get_table(table_ref)
    twenty_twenty_df = client.list_rows(table).to_dataframe()

    stations_df['STN'] = stations_df['usaf'] + '-' + stations_df['wban']
    twenty_twenty_df['STN'] = twenty_twenty_df['stn'] + '-' + twenty_twenty_df['wban']

    cols_1 = ['STN', 'mo', 'da', 'temp', 'min', 'max', 'stp', 'wdsp', 'prcp', 'fog']
    cols_2 = ['STN', 'country', 'state', 'call', 'lat', 'lon', 'elev']
    weather_df = twenty_twenty_df[cols_1].join(stations_df[cols_2].set_index('STN'), on='STN')

    weather_df.tail(10)

In [ ]:
df = pd.read_csv('../input/novel-corona-virus-2019-dataset/covid_19_data.csv',parse_dates=['ObservationDate','Last Update']).sort_values(by='ObservationDate').fillna('')
df.rename(columns={'ObservationDate':'Date', 'Country/Region':'Country', 'Province/State': 'Region'}, inplace=True)
df.drop(columns=['SNo', 'Deaths', 'Recovered'], inplace=True)
display(df.info())
display(df.sample(5))

print("External Data")
print(f"Earliest Entry: {df['Date'].min()}")
print(f"Last Entry:     {df['Date'].max()}")
print(f"Total Days:     {df['Date'].max() - df['Date'].min()}")

## Function to plot rate of change for grouping

In [ ]:
def plot_rate_of_change(df, country=None, region=None, grouping=['Date']):
    
    if country is None:
        selected_df = df
    elif region is None:
        selected_df = df.query('Country == "' + country + '"')
    else:
        selected_df = df.query('Country == "' + country + '" and Region == "' + region + '"' )
        
    country = 'Global' if country is None else country
    region = 'ALL' if region is None else region
    print('Country: {}'.format(country))
    print('Regions: {}'.format(region))

    grouped = selected_df.groupby(grouping).sum().reset_index().set_index('Date')
    grouped['Confirmed_ROC'] = grouped[['Confirmed']].pct_change().fillna(0) * 100.0
    grouped['SMA_3'] = grouped[['Confirmed_ROC']].rolling('3d').mean() 
    grouped['SMA_7'] = grouped[['Confirmed_ROC']].rolling('7d').mean()
    grouped['SMA_14'] = grouped[['Confirmed_ROC']].rolling('14d').mean()
    

    grouped[['Confirmed_ROC', 'SMA_3', 'SMA_7', 'SMA_14']].plot(title='Daily Rate of Change for: ' + country + ' - ' + region,
                                                       figsize=(20,8))
    plt.ylabel('pct%')   
    plt.show()
    display(pd.concat([grouped.head(1), grouped.tail(1)]))

plot_rate_of_change(df, 'US')
plot_rate_of_change(df, 'US', 'North Carolina')

## Global

In [ ]:
plot_rate_of_change(df)

In [ ]:
plot_rate_of_change(df, 'US')

In [ ]:
plot_rate_of_change(df, 'Italy')

In [ ]:
plot_rate_of_change(df, 'Mainland China')

In [ ]:
plot_rate_of_change(df, 'Hong Kong')

In [ ]:
plot_rate_of_change(df, 'South Korea')

In [ ]:
plot_rate_of_change(df, 'US', 'California')
plot_rate_of_change(df, 'US', 'New York')
plot_rate_of_change(df, 'US', 'North Carolina')
plot_rate_of_change(df, 'US', 'Ohio')
plot_rate_of_change(df, 'US', 'Florida')